# DIVAnd multivariate - EOF approach
This notebook shows a synthetic example of how the _multivariate_ analysis implemented in `DIVAnd`  works.<br>
Here the so-called EOF approach is used, while in [`4-11-multivariate-Jacobian`](4-11-multivariate-Jacobian.ipynb) the Jacobian approach is demonstrated.
## Packages

In [ ]:
import Pkg
Pkg.activate("../..")
Pkg.instantiate()
using Makie, CairoMakie
using LinearAlgebra
using Statistics
using DIVAnd

## Create synthetic observations

In [ ]:
ND = 159
NX = 200
NY = 250
NV = 2

# function to interpolate
fun(x, y, v) = 2 * (sin.(6x) * cos.(6y)) * (1.5 - v) .+ (v .- 1.0) .* x .* y

### Observations
The 2 first coordinates `x` and `y` can be considered as the _spatial_ coordinates.     
The 3rd coordinate `v` represent the variable that has been measured: 
- v[i] == __1__ means that at (x[i], y[i]), it is the _first_ variable that has been measured (for example, temperature).
- v[i] == __2__ means that the 2nd variable was measured at (x[i], y[i]).

In this simple example we will work with only 2 pseudo-variables, hence `v` only takes the values __1__ or __2__.

In [ ]:
x = 0.5 .+ 0.25 .* randn(ND);
y = 0.5 .+ 0.25 .* randn(ND);
v = mod.(rand(Int, ND), 2) .+ 1

# The locations of the observations are moved either to the left or to the right, 
# according to the variable that was measured.

x[v.>1.5] .+= 0.2
x[v.<1.5] .+= -0.2

# Observations
f = fun.(x, y, v) + 0.2 * randn(ND);

### Mask, grid and metrics
<div class="alert alert-block alert-info">
<b>Note:</b> the grid has now a 3rd dimension <code>vi</code>.
</div>

In [ ]:
# final grid
xi, yi, vi = ndgrid(range(0, stop = 1, length = NX), range(0, stop = 1, length = NY), 1:2);

# All points are valid points, except a rectangular area.
mask = trues(size(xi));
mask[10:30, 20:50, :] .= false

pm = ones(size(xi)) / (xi[2, 1, 1] - xi[1, 1, 1]);
pn = ones(size(xi)) / (yi[1, 2, 1] - yi[1, 1, 1]);
pv = ones(size(xi)) / (vi[1, 1, 2] - vi[1, 1, 1]);

# Reference field
fref = fun.(xi, yi, vi);

## Perform simple analysis
### Parameters
<div class="alert alert-block alert-info">
ℹ️ Again note that the correlation length has 3 components and that the 3rd is set to <b>0</b>.
</div>

In [ ]:
len = 0.2
epsilon2 = 1.0;
@time fi, s =
    DIVAndrun(mask, (pm, pn, pv), (xi, yi, vi), (x, y, v), f, (len, len, 0.0), epsilon2);

### Plot the interpolated field for each _variable_ and the corresponding observations

In [ ]:
fig = Figure(size = (900, 400))
ax = Axis(fig[1, 1], aspect = 1, title = "Variable #1")
hm = heatmap!(
    ax,
    xi[:, 1, 1],
    yi[1, :, 1],
    fi[:, :, 1],
    colorrange = [-1, 1],
    colormap = Reverse(:RdYlBu),
)
scatter!(ax, x[v.<1.5], y[v.<1.5], color = :black, markersize = 3)
xlims!(ax, 0, 1)
ylims!(ax, 0, 1)
Colorbar(fig[1, 2], hm)

ax2 = Axis(fig[1, 3], aspect = 1, title = "Variable #2")
hm2 = heatmap!(
    ax2,
    xi[:, 1, 1],
    yi[1, :, 1],
    fi[:, :, 2],
    colorrange = [-1, 1],
    colormap = Reverse(:RdYlBu),
)
scatter!(ax2, x[v.>1.5], y[v.>1.5], color = :black, markersize = 3)
Colorbar(fig[1, 4], hm2)
xlims!(ax2, 0, 1)
ylims!(ax2, 0, 1)
fig

## Perform multivariate analysis
### Transform correlation length to matrix

In [ ]:
len = 0.2 * ones(Float64, size(mask));

### Analysis
Check the function docstring to know more about the output of a `DIVAnd_multivarEOF` execution.

In [ ]:
@time fim, s, eofs, eofamplitudes, emap, emapm = DIVAnd_multivarEOF(
    mask,
    (pm, pn, pv),
    (xi, yi, vi),
    (x, y, v),
    f,
    (len, len, 0.0 .* len),
    epsilon2,
);

### Plot the 2 variables

In [ ]:
fig = Figure(size = (900, 400))
ax = Axis(fig[1, 1], aspect = 1, title = "Variable #1, EOF multivariate approach")
hm = heatmap!(
    ax,
    xi[:, 1, 1],
    yi[1, :, 1],
    fim[:, :, 1],
    colorrange = [-1, 1],
    colormap = Reverse(:RdYlBu),
)
scatter!(ax, x[v.<1.5], y[v.<1.5], color = :black, markersize = 3)
xlims!(ax, 0, 1)
ylims!(ax, 0, 1)
Colorbar(fig[1, 2], hm)

ax2 = Axis(fig[1, 3], aspect = 1, title = "Variable #2, EOF multivariate approach")
hm2 = heatmap!(
    ax2,
    xi[:, 1, 1],
    yi[1, :, 1],
    fim[:, :, 2],
    colorrange = [-1, 1],
    colormap = Reverse(:RdYlBu),
)
scatter!(ax2, x[v.>1.5], y[v.>1.5], color = :black, markersize = 3)
Colorbar(fig[1, 4], hm2)
xlims!(ax2, 0, 1)
ylims!(ax2, 0, 1)
fig

### Additional plots
#### Difference between the 2 analysis

In [ ]:
fig = Figure()
ax = Axis(
    fig[1, 1],
    aspect = 1,
    title = "Difference between simple and multivariate analysis\n(Variable #1)",
)
hm = heatmap!(
    ax,
    xi[:, 1, 1],
    yi[1, :, 1],
    fi[:, :, 1] .- fim[:, :, 1],
    colorrange = [-0.5, 0.5],
    colormap = Reverse(:RdBu),
)
scatter!(ax, x, y, color = :black, markersize = 3)
xlims!(ax, 0, 1)
ylims!(ax, 0, 1)
Colorbar(fig[1, 2], hm)
fig

#### EOF amplitude

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], aspect = 1, title = "EOF amplitude")
hm = heatmap!(
    ax,
    xi[:, 1, 1],
    yi[1, :, 1],
    eofamplitudes,
    colorrange = [-1.0, 1],
    colormap = Reverse(:RdYlBu),
)
xlims!(ax, 0, 1)
ylims!(ax, 0, 1)
Colorbar(fig[1, 2], hm)
fig

#### The 2 EOFSs

In [ ]:
fig = Figure(size = (900, 400))
ax = Axis(fig[1, 1], aspect = 1, title = "EOF #1")
hm = heatmap!(
    ax,
    xi[:, 1, 1],
    yi[1, :, 1],
    eofamplitudes * eofs[1],
    colorrange = [-1, 1],
    colormap = Reverse(:RdYlBu),
)
scatter!(ax, x[v.<1.5], y[v.<1.5], color = :black, markersize = 3)
xlims!(ax, 0, 1)
ylims!(ax, 0, 1)
Colorbar(fig[1, 2], hm)

ax2 = Axis(fig[1, 3], aspect = 1, title = "EOF #2")
hm2 = heatmap!(
    ax2,
    xi[:, 1, 1],
    yi[1, :, 1],
    eofamplitudes * eofs[2],
    colorrange = [-1, 1],
    colormap = Reverse(:RdYlBu),
)
scatter!(ax2, x[v.>1.5], y[v.>1.5], color = :black, markersize = 3)
Colorbar(fig[1, 4], hm2)
xlims!(ax2, 0, 1)
ylims!(ax2, 0, 1)
fig

#### Reference field

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], aspect = 1, title = "")
hm = heatmap!(
    ax,
    xi[:, 1, 1],
    yi[1, :, 1],
    fref[:, :, 2],
    colorrange = [-1.0, 1],
    colormap = Reverse(:RdYlBu),
)
xlims!(ax, 0, 1)
ylims!(ax, 0, 1)
Colorbar(fig[1, 2], hm)
fig

## Compute some statistics

In [ ]:
function nanvar(f)
    return var(f[.!isnan.(f)])
end
@show nanvar(fi - fref), nanvar(fim - fref)
@show nanvar(eofamplitudes * eofs[1] - fref[:, :, 1])
@show nanvar(fim[:, :, 1] - fref[:, :, 1])
@show nanvar(eofamplitudes * eofs[2] - fref[:, :, 2])
@show nanvar(fim[:, :, 2] - fref[:, :, 2])

In [ ]:
@show eof,
extrema(fref),
extrema(f),
extrema(fi[mask]),
extrema(fim[mask]),
extrema(eofamplitudes[mask[:, :, 1]] .* eofs[1]),
extrema(eofamplitudes[mask[:, :, 1]] .* eofs[2])

## Error fields

In [ ]:
fig = Figure(size = (1000, 300))
ax = Axis(fig[1, 1], aspect = 1, title = "Error field, variable #1, simple approach")
hm = heatmap!(
    ax,
    xi[:, 1, 1],
    yi[1, :, 1],
    emap[:, :, 1],
    colorrange = [0, 1],
    colormap = Reverse(:hot),
)
scatter!(ax, x[v.<1.5], y[v.<1.5], color = :black, markersize = 3)
xlims!(ax, 0, 1)
ylims!(ax, 0, 1)
Colorbar(fig[1, 2], hm)

ax2 = Axis(fig[1, 3], aspect = 1, title = "Error field, variable #2, simple approach")
hm2 = heatmap!(
    ax2,
    xi[:, 1, 2],
    yi[1, :, 2],
    emap[:, :, 2],
    colorrange = [0, 1],
    colormap = Reverse(:hot),
)
scatter!(ax2, x[v.>1.5], y[v.>1.5], color = :black, markersize = 3)
Colorbar(fig[1, 4], hm2)
xlims!(ax2, 0, 1)
ylims!(ax2, 0, 1)

ax3 = Axis(fig[1, 5], aspect = 1, title = "Error field, variable #1, multivariate approach")
hm3 = heatmap!(
    ax3,
    xi[:, 1, 1],
    yi[1, :, 1],
    emapm[:, :, 1],
    colorrange = [0, 1],
    colormap = Reverse(:hot),
)
scatter!(ax3, x[v.<1.5], y[v.<1.5], color = :black, markersize = 3)
Colorbar(fig[1, 6], hm3)
xlims!(ax3, 0, 1)
ylims!(ax3, 0, 1)

fig

### Comparison between the error fields

In [ ]:
@show nanvar(emap - emapm), nanvar(emap)

In [ ]:
fig = Figure(size = (900, 400))
ax = Axis(
    fig[1, 1],
    aspect = 1,
    title = "Variable #1, difference between simple\n and multivariate error field",
)
hm = heatmap!(
    ax,
    xi[:, 1, 1],
    yi[1, :, 1],
    emap[:, :, 1] - emapm[:, :, 1],
    colormap = Reverse(:RdBu),
)
xlims!(ax, 0, 1)
ylims!(ax, 0, 1)
Colorbar(fig[1, 2], hm)

ax2 = Axis(
    fig[1, 3],
    aspect = 1,
    title = "Variable #2, difference between simple\n and multivariate error field",
)
hm2 = heatmap!(
    ax2,
    xi[:, 1, 1],
    yi[1, :, 1],
    emap[:, :, 2] - emapm[:, :, 2],
    colormap = Reverse(:RdBu),
)
Colorbar(fig[1, 4], hm2)
xlims!(ax2, 0, 1)
ylims!(ax2, 0, 1)
fig